In [49]:
from modules import adquire, cleaning, create_df_checking, outputs
from numpy.random import choice
from time import sleep
#import pygame
#pygame.mixer.init()

In [50]:
#drumroll = pygame.mixer.Sound('drumroll.wav')
#drumroll.play()

def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df,hora= create_df_checking(df_students, df_katas)    
    outputs(df,False)
    return df


if __name__ == '__main__':
    df = main()

EduMartinezGarrido presentará la kata counting-sheep-dot-dot-dot


slug,counting-sheep-dot-dot-dot
username,
FranLRamos,True
capitanz,True
javielpapi,True
Darile,False
julistary,True
CristinaC,True
Unaiat,True
Marcunu,True
asiokfd,True


Alberto Pérez,asiokfd
Alberto Pérez Lescure,Albertoplm
Pablo Fernández,pablofdezc
Antonio Pita,Antoniopita95
Alejandro Solo,as-Solo
Pablo Fernández Romero,PabloFR
Carmen Rodríguez,Carmen-r
Susana Martín,susanamartin
Eduarfo Martínez,EduMartinezGarrido